# Bayesian Mixture Model

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.append('../')

import beer
import numpy as np

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [2]:
def plot_normal(fig, mean, cov, n_sigma=2, alpha=1., color='blue'):
    'Plot a Normal density'
    # Eigenvalue decomposition of the covariance matrix.
    evals, evecs = np.linalg.eigh(cov)
    
    sign = 1 if cov[1, 0] == 0 else np.sign(cov[1, 0])
    # Angle of the rotation.
    angle =  - np.arccos(sign * abs(evecs[0, 0]))
   
    fig.ellipse(x=mean[0], y=mean[1], 
                width=n_sigma * 2* np.sqrt(evals[0]), 
                height=n_sigma * 2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
    fig.ellipse(x=mean[0], y=mean[1], 
                width=2 * np.sqrt(evals[0]), 
                height=2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
def plot_mixture(fig, gmm, n_sigma=2, alpha=1., color='blue'):
    weights = gmm.weights
    for i, component in enumerate(gmm.components):
        plot_normal(fig, component.mean, component.cov, n_sigma, alpha * weights[i], color)

Generate some normally distributed data:

In [3]:
fig = figure(
    title='Data',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)

mean = np.array([-1.5, 4]) 
cov = np.array([
    [.75, 0],
    [0, 2.]
])
data1 = np.random.multivariate_normal(mean, cov, size=500)

plot_normal(fig, mean, cov, alpha=0.1)

mean = np.array([5, 5]) 
cov = np.array([
    [2, 1],
    [1, .75]
])
data2 = np.random.multivariate_normal(mean, cov, size=500)

plot_normal(fig, mean, cov, alpha=0.1)

data = np.vstack([data1, data2])
print(data.shape)
#np.random.shuffle(data)

fig.circle(data[:, 0], data[:, 1], alpha=.1)

show(fig)

(1000, 2)


In [16]:
p_mean = data.mean(axis=0)
p_cov = np.cov(data.T)
args = {'dim':2, 'mean':p_mean, 'cov': p_cov, 'prior_count':1e-6, 'random_init':True}
gmm_diag = beer.Mixture.create(10, beer.NormalDiagonalCovariance.create, args, prior_count=1e-6)
gmm_full = beer.Mixture.create(10, beer.NormalFullCovariance.create, args, prior_count=1e-6)
gmm_diag.fit(data, max_epochs=100)
gmm_full.fit(data, max_epochs=100)

In [17]:
fig1 = figure(
    title='GMM (diag)',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plot_mixture(fig1, gmm_diag, color='red')

fig2 = figure(
    title='GMM (full)',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plot_mixture(fig2, gmm_full, color='green')

grid = gridplot([[fig1, fig2]])
show(grid)

In [15]:
print(gmm_full.weights)
print(gmm_diag.weights)

[ 0.          0.          0.          0.49917365  0.50082635  0.          0.
  0.          0.          0.        ]
[ 0.00056421  0.16790491  0.28425399  0.10298374  0.01157743  0.07037486
  0.01900932  0.20554923  0.0005642   0.13721811]


In [ ]:
idxs = np.random.choice(101, size=101, replace=False)
np.array_split(idxs, 10)

In [ ]:
def mini_batches(data, mini_batch_size):
    indices = np.random.choice(data.shape[0], size=data.shape[0], replace=False)
    splits = np.array_split(indices, data.shape[0] // mini_batch_size)
    for split in splits:
        yield data[split]

In [ ]:
print(data.shape)

In [ ]:
for mini_batch in mini_batches(np.arange(20).reshape(10, -1), 3):
    print(mini_batch.shape, mini_batch)

In [ ]:
rng1 = np.random.RandomState()
rng2 = np.random.RandomState()
rng1.seed(1)
print(rng1.uniform(), rng2.uniform())

In [ ]:
x = np.arange(200).reshape(2, -1)
np.prod(x.shape[:-1])